<a href="https://colab.research.google.com/github/Palaeoprot/ModulAAR/blob/main/mainKinetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?export=view&id=1jmpFC9fmDMGKNuhGMdjMvKMXDOcW5iO2' width=700px align=centre>


MoDuLAAR is a tool for analysing amino acid racemization in protein diagenesis studies. It consists of several modules that work together to process data, perform analyses, and visualize results.

Sure! Based on the provided files and assuming they follow typical patterns, here is a markdown description for each notebook. This will help new users understand the purpose of each notebook and how they are used in the overall program.

---

## Overview of Notebooks

This program is composed of several interconnected notebooks, each designed to perform specific tasks related to the analysis of amino acid racemization and hydrolysis processes. Below is a brief overview of each notebook and its role in the program.

### 1. Data Processor (`data_processor.ipynb`)
The Data Processor notebook handles the preprocessing and cleaning of raw data. It ensures that the data is in the correct format and structure required for further analysis.

**Key Functions:**
- **Load Data**: Imports raw data from CSV files or Google Sheets.
- **Clean Data**: Cleans the data by handling missing values, correcting inconsistencies, and renaming columns for consistency.
- **Transform Data**: Converts data types and normalizes values as needed.

**Usage:**
This notebook is run first to prepare the raw data for subsequent analysis steps.

### 2. Dehydration Analyser (`dehydration_analyser.ipynb`)
The Dehydration Analyser notebook examines the effects of dehydration on amino acid concentrations and D/L ratios. It processes data to identify trends and patterns related to dehydration.

**Key Functions:**
- **Analyze Dehydration**: Processes data to evaluate the impact of dehydration on the samples.
- **Visualize Results**: Generates plots and charts to visualize dehydration effects.

**Usage:**
Run this notebook after data processing to analyze dehydration-related phenomena.

### 3. Racemization Simulator (`racemization_simulator.ipynb`)
The Racemization Simulator notebook simulates the kinetics of hydrolysis and racemization processes. It models how these processes affect amino acid concentrations and D/L ratios over time.

**Key Functions:**
- **Simulate Kinetics**: Runs simulations to model hydrolysis and racemization.
- **Calculate Ratios**: Computes D/L ratios based on simulation results.
- **Plot Results**: Visualizes the simulation outcomes through various plots.

**Usage:**
Use this notebook to simulate and understand the behavior of amino acids under different conditions.

### 4. Parameter optimizer (`parameter_optimizer.ipynb`)
The Parameter optimizer notebook optimizes the parameters used in the racemization and hydrolysis simulations. It ensures that the model parameters are tuned for accuracy.

**Key Functions:**
- **optimize Parameters**: Utilizes optimization algorithms to find the best-fit parameters for the model.
- **Evaluate Model**: Assesses the model's performance and adjusts parameters accordingly.

**Usage:**
Run this notebook to fine-tune the parameters before performing detailed simulations.

### 5. Result Visualiser (`result_visualiser.ipynb`)
The Result Visualiser notebook focuses on presenting the results of the analyses and simulations in an understandable format. It generates comprehensive visualizations to aid in interpreting the data.

**Key Functions:**
- **Visualize Data**: Creates detailed plots and charts to display the analysis and simulation results.
- **Generate Reports**: Compiles the visualizations into reports for easier interpretation.

**Usage:**
Use this notebook to generate and view visual representations of the processed and simulated data.

### Main Notebook (`main.ipynb`)
The main notebook orchestrates the execution of the other notebooks. It ensures that each step is performed in the correct order, starting from data processing to final visualization.

**Key Functions:**
- **Run Notebooks**: Sequentially runs each of the other notebooks using `%run` commands.
- **Manage Workflow**: Ensures that data flows correctly from one notebook to the next, maintaining the integrity of the analysis.

**Usage:**
Execute the main notebook to run the entire program from start to finish. This provides a streamlined process for analysing amino acid racemization and hydrolysis.



In [ ]:
!pip install nbimporter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nbformat
from IPython import get_ipython

In [ ]:
import nbformat
from IPython import get_ipython
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

R = 8.314  # Gas constant in J/(mol·K)

def execute_notebook(notebook_path):
    """Execute a Jupyter notebook and run its code cells."""
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    ip = get_ipython()
    global_namespace = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                exec(cell.source, global_namespace)
            except Exception as e:
                print(f"Error executing cell: {e}")

    return global_namespace

def arrhenius_plot(temps, rate_constants, title, ylabel):
    """Generate an Arrhenius plot."""
    inverse_temps = 1 / (temps + 273.15)  # Convert to Kelvin and take the inverse
    log_rate_constants = np.log(rate_constants)

    plt.figure()
    plt.plot(inverse_temps, log_rate_constants, 'o')
    plt.xlabel('1 / Temperature (1/K)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

def main():
    # Path to the hydrolysis and decomposition estimates notebook
    notebook_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/hydrolysis_decomposition_estimates.ipynb'

    # Execute the hydrolysis_decomposition_estimates notebook
    try:
        print(f"Executing {notebook_path}...")
        global_namespace = execute_notebook(notebook_path)
        print(f"Successfully executed {notebook_path}")
    except Exception as e:
        print(f"Failed to execute {notebook_path}: {str(e)}")

    # Verify if the functions are defined
    try:
        optimise_rates = global_namespace['optimise_rates']
        plot_results = global_namespace['plot_results']
        print("Successfully accessed functions from hydrolysis_decomposition_estimates notebook")
    except KeyError:
        print("Failed to access functions - they may not be defined.")
        optimise_rates = None
        plot_results = None

    if optimise_rates and plot_results:
        # Load the aggregated data
        data_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ProcessedData/real_DL_summed_totals.csv'
        data = pd.read_csv(data_path)

        # Check if data is loaded correctly
        if data.empty:
            print("Data is empty. Please check the data file.")
        else:
            print("Data loaded successfully.")
            print(data.head())

        # Optimize rates
        try:
            optimized_parameters = optimise_rates(data)

            # Plot the results
            plot_results(data, optimized_parameters)

            # Arrhenius plots
            temperatures = data['temp (°C)'].unique()
            k_h_values = [params[1] for params in optimized_parameters]
            k_d_values = [params[2] for params in optimized_parameters]
            arrhenius_plot(temperatures, k_h_values, 'Arrhenius Plot for Hydrolysis', 'ln(k_h)')
            arrhenius_plot(temperatures, k_d_values, 'Arrhenius Plot for Decomposition', 'ln(k_d)')

        except Exception as e:
            print(f"Failed to optimize rates or plot results: {str(e)}")
    else:
        print("Skipping optimization and plotting due to function access issues")

if __name__ == '__main__':
    main()

In [ ]:
# import nbformat
# from IPython import get_ipython
# import pandas as pd
# import numpy as np

# def execute_notebook(notebook_path):
#     """Execute a Jupyter notebook and run its code cells."""
#     with open(notebook_path, 'r', encoding='utf-8') as f:
#         nb = nbformat.read(f, as_version=4)

#     ip = get_ipython()
#     global_namespace = {}
#     for cell in nb.cells:
#         if cell.cell_type == 'code':
#             try:
#                 exec(cell.source, global_namespace)
#             except Exception as e:
#                 print(f"Error executing cell: {e}")

#     return global_namespace

# # Path to the hydrolysis and decomposition estimates notebook
# notebook_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/hydrolysis_decomposition_estimates.ipynb'

# # Execute the hydrolysis_decomposition_estimates notebook
# try:
#     print(f"Executing {notebook_path}...")
#     global_namespace = execute_notebook(notebook_path)
#     print(f"Successfully executed {notebook_path}")
# except Exception as e:
#     print(f"Failed to execute {notebook_path}: {str(e)}")

# # Verify if the functions are defined
# try:
#     optimise_rates = global_namespace['optimise_rates']
#     plot_results = global_namespace['plot_results']
#     print("Successfully accessed functions from hydrolysis_decomposition_estimates notebook")
# except KeyError:
#     print("Failed to access functions - they may not be defined.")
#     optimise_rates = None
#     plot_results = None

# if optimise_rates and plot_results:
#     # Load the aggregated data
#     data_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ProcessedData/real_DL_summed_totals.csv'
#     data = pd.read_csv(data_path)

#     # Check if data is loaded correctly
#     if data.empty:
#         print("Data is empty. Please check the data file.")
#     else:
#         print("Data loaded successfully.")
#         print(data.head())

#     # Optimize rates
#     try:
#         optimized_parameters = optimise_rates(data)
#         # Plot the results
#         plot_results(data, optimized_parameters)
#     except Exception as e:
#         print(f"Failed to optimize rates or plot results: {str(e)}")
# else:
#     print("Skipping optimization and plotting due to function access issues.")
